In [18]:
import csv
import numpy as np
import time

file_path = 'files/filtered_feature_table.csv'
data = []
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        del row[0]
        data.append(row)
del data[0]
del data[50:]
for i in range(len(data)):
    for j in range(len(data[i])):
        if float(data[i][j]) > 0:
            data[i][j] = 1
        else:
            data[i][j] = 0

matrix_size = len(data)
mat = np.array(data)
matriz_adj = np.zeros((matrix_size, matrix_size))

for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if np.sum(mat[i] & mat[j]) >= 3:
            matriz_adj[i][j] = 1
            matriz_adj[j][i] = 1

N = {
    i: set(num for num, j in enumerate(row) if j)
    for i, row in enumerate(matriz_adj)
}

def BronKerbosch1(P, R=None, X=None):
    P = set(P)
    R = set() if R is None else R
    X = set() if X is None else X
    if not P and not X:
        yield R
    while P:
        v = P.pop()
        yield from BronKerbosch1(
            P=P.intersection(N[v]), R=R.union([v]), X=X.intersection(N[v]))
        X.add(v)

# Executando o algoritmo de Bron–Kerbosch e medindo o tempo de execução
P = N.keys()
start_time = time.time()
cliques = list(BronKerbosch1(P))
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time} segundos")

# Salvando os resultados em um arquivo CSV
result_file_path = 'files/cliques_resultados.csv'
with open(result_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(cliques)

print(f"Resultados salvos em: {result_file_path}")


KeyboardInterrupt: 

In [5]:
import csv
import numpy as np

file_path = 'files/feature_table.csv'
data = []
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        del row[0]
        data.append(row)
del data[0]
del data[100:]
for row in data:
    del row[0]

for i in range(len(data)):
    for j in range(len(data[i])):
        if float(data[i][j]) > 0:
            data[i][j] = 1
        else:
            data[i][j] = 0

matrix_size = len(data)
mat = np.array(data)
matriz_adj = np.zeros((matrix_size, matrix_size))

for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if np.sum(mat[i] & mat[j]) >= 3:
            matriz_adj[i][j] = 1
            matriz_adj[j][i] = 1

N = {
    i: set(num for num, j in enumerate(row) if j)
    for i, row in enumerate(matriz_adj)
}

def BronKerbosch1(P, R=None, X=None):
    P = set(P)
    R = set() if R is None else R
    X = set() if X is None else X
    if not P and not X:
        return [list(R)]
    cliques = []
    while P:
        v = P.pop()
        new_R = R.union([v])
        new_P = P.intersection(N[v])
        new_X = X.intersection(N[v])
        cliques.extend(BronKerbosch1(new_P, new_R, new_X))
        X.add(v)
    return cliques

P = N.keys()
cliques = list(BronKerbosch1(P))

# Salvando os cliques em um arquivo CSV
output_file_path = 'files/cliques_result.csv'
with open(output_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(cliques)

print(f"Os cliques foram armazenados em {output_file_path}")


Os cliques foram armazenados em files/cliques_result.csv


In [16]:
import csv
import numpy as np
import time

def bron_kerbosch(graph, clique, candidates, excluded):
    if not candidates and not excluded:
        return clique

    max_clique = []
    for vertex in sorted(list(candidates), key=lambda v: len(graph[v]), reverse=True):
        neighbors = set(graph[vertex])
        current_clique = bron_kerbosch(
            graph,
            clique + [vertex],
            candidates.intersection(neighbors),
            excluded.intersection(neighbors),
        )
        if len(current_clique) > len(max_clique):
            max_clique = current_clique

        candidates.remove(vertex)
        excluded.add(vertex)

    return max_clique

file_path = 'files/feature_table.csv'

# Leitura do arquivo CSV e criação da matriz de adjacência
data = []
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        del row[0]
        data.append(row)
del data[0]
del data[50:]
for row in data:
    del row[0]

# Remoção das linhas que contêm menos de 5 valores diferentes de 0
filtered_data = [row for row in data if sum(1 for value in row[1:] if float(value) != 0) >= 5]

for i in range(len(data)):
    for j in range(len(data[i])):
        if float(data[i][j]) > 0:
            data[i][j] = 1
        else:
            data[i][j] = 0

matrix_size = len(data)
mat = np.array(data)
matriz_adj = np.zeros((matrix_size, matrix_size))

for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if np.sum(mat[i] & mat[j]) >= 3:
            matriz_adj[i][j] = 1
            matriz_adj[j][i] = 1

# Convertendo a matriz de adjacência para um formato de dicionário de grafos
graph_dict = {i: np.nonzero(row)[0].tolist() for i, row in enumerate(matriz_adj)}

# Aplicação do algoritmo de Bron–Kerbosch com heurísticas
start_time = time.time()
result_clique = bron_kerbosch(graph_dict, [], set(graph_dict.keys()), set())
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time} segundos")

# Salvando os resultados em um arquivo CSV
result_file_path = 'files/bron_kerbosch_results.csv'
with open(result_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Vertex", "Clique Size"])
    for vertex in result_clique:
        csv_writer.writerow([vertex, len(result_clique)])

print(f"Resultados salvos em: {result_file_path}")


Tempo de execução: 0.0005676746368408203 segundos
Resultados salvos em: files/bron_kerbosch_results.csv
